# Part 1

Write a function that estimates the emission parameters from the training set using MLE (maximum
likelihood estimation):

In [14]:
def read_file(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        file_content = file.read()
    return file_content

def write_file(file_path, content):
    with open(file_path, 'w', encoding="utf-8") as file:
        file.write(content)

In [2]:
def estimate_emission_parameters(file_content):
    tags = {}  # Dictionary to store counts of each observation
    emission_params = {}  # Dictionary to store emission probabilities

    # Split the file content into lines
    lines = file_content.strip().split("\n")

    # Iterate through each line and extract observations and tags
    for line in lines:
        if len(line) == 0:
            continue
        if line[-1] == 'O':
            observation, tag = line[:-1], line[-1]
        elif ('B-positive' in line 
            or 'I-positive' in line 
            or 'B-negative' in line 
            or 'I-negative' in line):
            observation, tag = line[:-10], line[-10:]
        elif ('B-neutral' in line
            or 'I-neutral' in line):
            observation, tag = line[:-9], line[-9:]
#         observation, tag = line.split()
    
        if tag in tags:
            tags[tag][observation] = tags[tag].get(observation, 0) + 1
        else:
            tags[tag] = {observation: 1}

    # Calculate emission probabilities for each observation and tag
    for tag, observations_count in tags.items():
        total_count = sum(observations_count.values())
        emission_params[tag] = {
            observation: count / total_count for observation, count in observations_count.items()
        }

    return emission_params

file_content = read_file("ES/train.txt")

emission_parameters = estimate_emission_parameters(file_content)
# print(emission_parameters, sum(len(words) for words in emission_parameters.values()))

One problem with estimating the emission parameters is that some words that appear in the test set
do not appear in the training set. One simple idea to handle this issue is as follows. We introduce
a special word token #UNK#, and make the following modifications to the computation of emission
probabilities.During the testing phase, if the word does not appear in the training set, we replace that word with
#UNK#. Set k to 1, implement this fix into your function for computing the emission parameters.

In [29]:
def estimate_emission_parameters_modified(training_file_content, test_file_content, k=1):
    tags = {}  # Dictionary to store counts of each observation
    emission_params = {}  # Dictionary to store emission probabilities
    train_words = []
    test_words = []
    
    # Split the file content into lines
    train_data_lines = training_file_content.strip().split("\n")
    test_data_lines = test_file_content.strip().split("\n")

    # Iterate through each line and extract observations and tags
    for line in train_data_lines:
        if len(line) == 0:
            continue
        if ' O' in line:
            observation, tag = line[:-2], line[-1]
        elif ('B-positive' in line 
            or 'I-positive' in line 
            or 'B-negative' in line 
            or 'I-negative' in line):
            observation, tag = line[:-11], line[-10:]
        elif ('B-neutral' in line
            or 'I-neutral' in line):
            observation, tag = line[:-10], line[-9:]
        if observation not in train_words:
            train_words.append(observation)
        if tag in tags:
            tags[tag][observation] = tags[tag].get(observation, 0) + 1
        else:
            tags[tag] = {observation: 1}
    # Iterate through each line to extract observations from test set
    for line in test_data_lines:
        if len(line) == 0:
            continue
        if ' O' in line:
            observation, tag = line[:-2], line[-1]
        elif ('B-positive' in line 
            or 'I-positive' in line 
            or 'B-negative' in line 
            or 'I-negative' in line):
            observation, tag = line[:-11], line[-10:]
        elif ('B-neutral' in line
            or 'I-neutral' in line):
            observation, tag = line[:-10], line[-9:]
        else:
            observation = line
        if observation not in test_words:
            test_words.append(observation)
    # Extract words that are in test set but not in train set   
    unique_words = find_unique_words(test_words, train_words)

    # Calculate emission probabilities for each observation and tag
    for tag, observations_count in tags.items():
        total_count = sum(observations_count.values())
        emission_params[tag] = {
            observation: count / (total_count + k) for observation, count in observations_count.items()
        }
        for word in unique_words:
            emission_params[tag][word] = k / (total_count + k)


    return emission_params, list(set(test_words).union(train_words))

def find_unique_words(list1, list2):
    set1 = set(list1)
    set2 = set(list2)

    unique_in_list1 = set1 - set2

    return list(unique_in_list1)


training_file_content = read_file("ES/train.txt")
test_file_content = read_file("ES/dev.in")
emission_parameters = estimate_emission_parameters_modified(training_file_content, test_file_content, 1)[0]
print(emission_parameters, sum(len(words) for words in emission_parameters.values()))
count = 0
for emission_probabilities in emission_parameters.values():
    count += len(emission_probabilities)
# print(count)

{'O': {'Estuvimos': 0.00020664003306240529, 'hace': 0.0008954401432704229, 'poco': 0.0018942003030720485, 'mi': 0.0024796803967488635, 'pareja': 0.00044772007163521146, 'y': 0.0352665656426505, 'yo': 0.0012398401983744318, 'comiendo': 0.00034440005510400884, 'resultó': 0.00013776002204160352, 'todo': 0.0039606006336961016, 'muy': 0.013638242182118749, 'bien': 0.005682600909216145, ',': 0.05730816916930707, 'tanto': 0.0013431602149056344, 'la': 0.026002204160352666, 'el': 0.022110483537677365, '…': 0.0015498002479680396, 'nos': 0.005028240804518529, 'gustó': 0.0003788400606144097, 'mucho': 0.0018253202920512468, '.': 0.055896128943380634, 'Por': 0.0010332001653120264, 'poner': 0.00017220002755200442, 'algún': 0.00020664003306240529, 'pero': 0.006578041052486569, 'quizá': 0.00013776002204160352, 'no': 0.012708362033337925, 'era': 0.0017908802865408459, 'lo': 0.009298801487808239, '"': 0.0013431602149056344, 'ibérico': 6.888001102080176e-05, 'que': 0.029101804656288744, 'cabía': 3.4440005

In [30]:
def sentiment_analysis(path_to_train_file, path_to_test_file):
    training_file_content = read_file(path_to_train_file)
    test_file_content = read_file(path_to_test_file)
    emission_parameters, words = estimate_emission_parameters_modified(training_file_content, test_file_content)
    word_to_label = {}
    for word in words:
        probabilities = []
        for label, freqs in emission_parameters.items():
            if word in freqs:
                probabilities.append((label, freqs[word]))
        word_to_label[word] = max(probabilities, key=lambda x: x[1])[0]
    return word_to_label

def write_result_to_file(word_to_label, path_to_dev_set, path_to_output):
    result = ""
    dev_file_content = read_file(path_to_dev_set)
    dev_set_lines = dev_file_content.strip().split("\n")
    counter = 0
    for line in dev_set_lines:
        if line == "":
            result += "\n"
            continue  
        result += line + " " + word_to_label.get(line) + "\n"
 
    write_file(path_to_output, result)        

In [31]:
mapping = sentiment_analysis("ES/train.txt", "ES/dev.in")
write_result_to_file(mapping, "ES/dev.in", "ES/dev.p1.out")

In [32]:
mapping = sentiment_analysis("RU/train.txt", "RU/dev.in")
write_result_to_file(mapping, "RU/dev.in", "RU/dev.p1.out")

## ES
#Entity in gold data: 229
#Entity in prediction: 1466

#Correct Entity : 178
Entity  precision: 0.1214
Entity  recall: 0.7773
Entity  F: 0.2100

#Correct Sentiment : 97
Sentiment  precision: 0.0662
Sentiment  recall: 0.4236
Sentiment  F: 0.1145

## RU
#Entity in gold data: 389
#Entity in prediction: 1816

#Correct Entity : 266
Entity  precision: 0.1465
Entity  recall: 0.6838
Entity  F: 0.2413

#Correct Sentiment : 129
Sentiment  precision: 0.0710
Sentiment  recall: 0.3316
Sentiment  F: 0.1170

# Part 2

Write a function that estimates the transition parameters from the training set using MLE (maximum
likelihood estimation):

In [19]:
def estimate_transition_parameters(file_content):
    transition_counts = {}
    transition_params = {}  # Dictionary to store emission probabilities

    # Split the file content into lines
    lines = file_content.strip().split("\n")

    # Iterate through each line and extract observations and tags
    for line_index in range(len(lines)):
        next_line = ""
        curr_line = lines[line_index]
        if line_index < len(lines) - 1:
            next_line = lines[line_index + 1]
        
        if len(curr_line) == 0:
            continue
        if curr_line[-1] == 'O':
            curr_observation, curr_tag = curr_line[:-1], curr_line[-1]
        elif ('B-positive' in curr_line 
            or 'I-positive' in curr_line 
            or 'B-negative' in curr_line 
            or 'I-negative' in curr_line):
            curr_observation, curr_tag = curr_line[:-10], curr_line[-10:]
        elif ('B-neutral' in curr_line
            or 'I-neutral' in curr_line):
            curr_observation, curr_tag = curr_line[:-9], curr_line[-9:]
        
        if len(next_line) == 0: #true if line_index = len(lines - 1) or line_index is last index of a document
            pass
        elif next_line[-1] == 'O':
            next_observation, next_tag = next_line[:-1], next_line[-1]
        elif ('B-positive' in next_line 
            or 'I-positive' in next_line 
            or 'B-negative' in next_line 
            or 'I-negative' in next_line):
            next_observation, next_tag = next_line[:-10], next_line[-10:]
        elif ('B-neutral' in next_line
            or 'I-neutral' in next_line):
            next_observation, next_tag = next_line[:-9], next_line[-9:]        
        
        # Handle transitions from START
        if line_index == 0 or lines[line_index - 1] == "":
            if 'START' in transition_counts:
                transition_counts['START'][curr_tag] = transition_counts['START'].get(curr_tag, 0) + 1
            else:
                transition_counts['START'] = {curr_tag: 1}
                
        # Handle transitions to STOP
        if len(next_line) == 0:
            if curr_tag in transition_counts:
                transition_counts[curr_tag]['STOP'] = transition_counts[curr_tag].get('STOP', 0) + 1
            else:
                transition_counts[curr_tag] = {'STOP': 1}
                
        # Rest of the transitions
        else:
            if curr_tag in transition_counts:
                transition_counts[curr_tag][next_tag] = transition_counts[curr_tag].get(next_tag, 0) + 1
            else:
                transition_counts[curr_tag] = {next_tag: 1}
        

    # Calculate transition probabilities 
    for tag, next_tag_counts in transition_counts.items():
        total_count = sum(next_tag_counts.values())
        transition_params[tag] = {
            next_tag: count / total_count for next_tag, count in next_tag_counts.items()
        }

    return transition_params

file_content = read_file("ES/train.txt")

transition_parameters = estimate_transition_parameters(file_content)
print(transition_parameters)

{'START': {'O': 0.9289176090468497, 'B-positive': 0.052234787291330104, 'B-negative': 0.014001077005923533, 'B-neutral': 0.004846526655896607}, 'O': {'O': 0.8856896848630963, 'B-positive': 0.03650766316514551, 'STOP': 0.06344067504735663, 'B-negative': 0.012226623041157224, 'B-neutral': 0.0021353538832443605}, 'B-positive': {'O': 0.871551724137931, 'I-positive': 0.11637931034482758, 'STOP': 0.008620689655172414, 'B-neutral': 0.0008620689655172414, 'B-positive': 0.002586206896551724}, 'B-negative': {'O': 0.8110236220472441, 'I-negative': 0.1784776902887139, 'STOP': 0.010498687664041995}, 'B-neutral': {'I-neutral': 0.20833333333333334, 'O': 0.7916666666666666}, 'I-neutral': {'I-neutral': 0.6511627906976745, 'O': 0.3488372093023256}, 'I-positive': {'I-positive': 0.5700636942675159, 'O': 0.4267515923566879, 'STOP': 0.0031847133757961785}, 'I-negative': {'O': 0.39766081871345027, 'I-negative': 0.6023391812865497}}


In [27]:
def viterbi(observation_sequence, emission_params, transition_params):
    states = list(emission_params.keys())
    T = len(observation_sequence)
    N = len(states)

    # Initialization step
    viterbi_table = [{}]
    backpointer = [{}]
    for state in states:
        viterbi_table[0][state] = transition_params['START'].get(state, 0) * emission_params[state].get(observation_sequence[0], 0)
        backpointer[0][state] = None

    # Recursion step
    for t in range(1, T):
        viterbi_table.append({})
        backpointer.append({})
        for state in states:
            max_prob = max(
                viterbi_table[t - 1][prev_state] * transition_params[prev_state].get(state, 0) * emission_params[state].get(observation_sequence[t], 0)
                for prev_state in states
            )
            viterbi_table[t][state] = max_prob
            backpointer[t][state] = max(states, key=lambda prev_state: viterbi_table[t - 1][prev_state] * transition_params[prev_state].get(state, 0))

    # Termination step
    max_prob_last_state = max(viterbi_table[T - 1].values())
    best_last_state = max(states, key=lambda state: viterbi_table[T - 1][state] * transition_params[state].get('STOP', 0))

    # Backtrack to find the best sequence
    best_sequence = [best_last_state]
    prev_state = best_last_state
    for t in range(T - 2, -1, -1):
        best_sequence.insert(0, backpointer[t + 1][prev_state])
        prev_state = backpointer[t + 1][prev_state]

    return best_sequence

def create_observation_sequences(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        file_content = file.read().strip()

    observation_sequences = []
    current_sequence = []

    # Split the file content by empty lines to get individual sequences
    sequence_lines = file_content.split("\n\n")

    for sequence_line in sequence_lines:
        # Split each sequence into individual observations (words)
        observation_sequence = sequence_line.strip().split("\n")
        current_sequence.extend(observation_sequence)
        observation_sequences.append(current_sequence)
        current_sequence = []

    return observation_sequences

def viterbi_for_sequences(observation_sequences, emission_params, transition_params):
    best_sequences = []
    for observation_sequence in observation_sequences:
        best_sequence = viterbi(observation_sequence, emission_params, transition_params)
        best_sequences.append(best_sequence)

    return best_sequences

def write_sequences_to_file(output_file_path, observation_sequences, best_sequences):
    with open(output_file_path, 'w', encoding='utf-8') as file:
        for obs_sequence, best_sequence in zip(observation_sequences, best_sequences):
            for word, tag in zip(obs_sequence, best_sequence):
                file.write(f"{word} {tag}\n")
            file.write("\n")  # Separate sequences with an empty line

In [ ]:
training_file_content = read_file("ES/train.txt")
test_file_content = read_file("ES/dev.in")
emission_params = estimate_emission_parameters_modified(training_file_content, test_file_content, 1)[0]
transition_params = estimate_transition_parameters(training_file_content)
observation_sequences = create_observation_sequences("ES/dev.in")
best_sequences = viterbi_for_sequences(observation_sequences, emission_params, transition_params)
# print(best_sequences)
   
write_sequences_to_file("ES/dev.p2.out", observation_sequences, best_sequences)

In [28]:
training_file_content = read_file("RU/train.txt")
test_file_content = read_file("RU/dev.in")
emission_params = estimate_emission_parameters_modified(training_file_content, test_file_content, 1)[0]
transition_params = estimate_transition_parameters(training_file_content)
observation_sequences = create_observation_sequences("RU/dev.in")
best_sequences = viterbi_for_sequences(observation_sequences, emission_params, transition_params)
# print(best_sequences)
   
write_sequences_to_file("RU/dev.p2.out", observation_sequences, best_sequences)

## ES
#Entity in gold data: 229
#Entity in prediction: 542

#Correct Entity : 134
Entity  precision: 0.2472
Entity  recall: 0.5852
Entity  F: 0.3476

#Correct Sentiment : 97
Sentiment  precision: 0.1790
Sentiment  recall: 0.4236
Sentiment  F: 0.2516

## RU

#Entity in gold data: 389
#Entity in prediction: 514

#Correct Entity : 190
Entity  precision: 0.3696
Entity  recall: 0.4884
Entity  F: 0.4208

#Correct Sentiment : 129
Sentiment  precision: 0.2510
Sentiment  recall: 0.3316
Sentiment  F: 0.2857